In [20]:
import sys
sys.path.append('..')
from utils.redis_utils import * 
from utils.postgres_utils import *

redis_client = RedisUtils(RedisType.CLUSTER).redis_client
postges_utils = PostgresUtils()

MAX_CACHE_SIZE = 50000

def fetch_data_from_postgres(product_id):
    print("Fetching data from PostgreSQL...")
    postges_utils.fetch_data_from_postgres(product_id)

def get_product_info(product_id):
    cached_data = redis_client.get(f"product:{product_id}")
    if cached_data:
        print("Data found in Redis cache!")
        return cached_data.decode('utf-8')
    
    data = fetch_data_from_postgres(product_id)

    redis_client.setex(f"product:{product_id}", 3600, str(data))

    if redis_client.dbsize() > MAX_CACHE_SIZE:
        oldest_key = redis_client.execute_command("LINDEX", 'product_keys', -1)
        redis_client.delete(oldest_key)
        redis_client.execute_command("LPOP", "product_keys")
    
    redis_client.rpush("product_keys", f"product:{product_id}")

    return data

def invalidate_product_cache(product_id):
    redis_client.delete(f"product:{product_id}")
    redis_client.lrem("product_keys", 0, f"product:{product_id}")
    print(f"cache for product {product_id} invalidated.")

def add_new_product(name, price, code):
    new_product_id = postges_utils.add_new_product(name, price, code)
    invalidate_product_cache(new_product_id)
    print(f"New product added with ID {new_product_id}.")

    return new_product_id

def update_product_price(product_id, new_price):
    postges_utils.update_product_price(product_id, new_price)
    invalidate_product_cache(product_id)
    print(f"Product {product_id} price updated.")

def fetch_random_product_with_cache(num_products_to_fetch):
    start_time = time.time()
    for _ in range(num_products_to_fetch):
        product_id = random.randint(1, num_products_to_fetch)
        _ = get_product_info(product_id)
    end_time = time.time()
    return float(f"{end_time - start_time:.2f}")

def fetch_random_product_without_cache(num_products_to_fetch):
    start_time = time.time()
    for _ in range(num_products_to_fetch):
        product_id = random.randint(1, num_products_to_fetch)
        _ = fetch_data_from_postgres(product_id)
    end_time = time.time()
    return float(f"{end_time - start_time:.2f}")


In [26]:
time_fetch_with_cache = fetch_random_product_with_cache(10000)
time_fetch_without_cache = fetch_random_product_without_cache(10000)
print("Time with cache:", time_fetch_with_cache, "seconds")
print("Time without cache:", time_fetch_without_cache, "seconds")

Data found in Redis cache!
Data found in Redis cache!
Data found in Redis cache!
Data found in Redis cache!
Data found in Redis cache!
Data found in Redis cache!
Data found in Redis cache!
Data found in Redis cache!
Data found in Redis cache!
Data found in Redis cache!
Data found in Redis cache!
Data found in Redis cache!
Data found in Redis cache!
Data found in Redis cache!
Data found in Redis cache!
Data found in Redis cache!
Data found in Redis cache!
Data found in Redis cache!
Data found in Redis cache!
Data found in Redis cache!
Data found in Redis cache!
Data found in Redis cache!
Data found in Redis cache!
Data found in Redis cache!
Data found in Redis cache!
Data found in Redis cache!
Data found in Redis cache!
Data found in Redis cache!
Data found in Redis cache!
Data found in Redis cache!
Data found in Redis cache!
Data found in Redis cache!
Data found in Redis cache!
Data found in Redis cache!
Data found in Redis cache!
Data found in Redis cache!
Data found in Redis cache!
D